In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [4]:
SELECT med.nomeMedicamento, COUNT(*) Qnt
    FROM Medicamento med, UsoMedicamento uso
    WHERE med.Codigo = uso.CodMedicamento
    GROUP BY med.nomeMedicamento;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [5]:
CREATE VIEW MaisUsado AS
    SELECT med.Codigo, COUNT(*) QNT
    FROM Medicamento med, UsoMedicamento uso
    WHERE med.Codigo = uso.CodMedicamento
    GROUP BY med.Codigo;

In [6]:
SELECT * FROM MaisUsado
    ORDER BY QNT DESC
    LIMIT 1;

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [7]:
SELECT CodMedicamento, Count(*) QNT
    FROM UsoMedicamento
    GROUP BY CodMedicamento
    ORDER BY QNT DESC
    LIMIT 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [8]:
SELECT med.nomeMedicamento, COUNT(*) Qnt
    FROM Medicamento med, UsoMedicamento uso
    WHERE med.Codigo = uso.CodMedicamento
    GROUP BY med.nomeMedicamento
    ORDER BY Qnt DESC
    LIMIT 1;

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [9]:
CREATE VIEW MediaAux AS
    SELECT IDpessoa, COUNT(*) Qnt
    FROM UsoMedicamento
    GROUP BY IDpessoa;
SELECT AVG(Qnt) FROM MediaAux;

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [10]:
SELECT IDpessoa
    FROM MediaAux
    WHERE Qnt > (SELECT AVG(Qnt)
                 FROM MediaAux);

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [11]:
SELECT Idade, AVG(Qnt)
    FROM (SELECT P.Idade Idade, COUNT(*) Qnt
         FROM Pessoa P, UsoMedicamento U
         WHERE P.ID = U.IDpessoa
         GROUP BY IDpessoa)
    GROUP BY Idade;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [12]:
CREATE VIEW medicamentoA AS
    SELECT IDpessoa, CodMedicamento MedicamentoA
    FROM UsoMedicamento;

CREATE VIEW EfeitoCruzado AS
    SELECT A.IDpessoa, A.MedicamentoA, B.CodMedicamento MedicamentoB
    FROM MedicamentoA A, UsoMedicamento B
    WHERE A.IDpessoa = B.IDpessoa
    AND A.MedicamentoA < B.CodMedicamento;

SELECT * FROM EfeitoCruzado;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [13]:
SELECT MedicamentoA, MedicamentoB, COUNT(*) QntPessoas
    FROM EfeitoCruzado
    GROUP BY MedicamentoA, MedicamentoB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [14]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [16]:
UPDATE Matriz
    SET IdadeAte60 = True
    WHERE Idade <= 60;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [18]:
UPDATE Matriz
    SET MaisUmAnoMedicamento = True
    WHERE ID IN (SELECT DISTINCT IdPessoa
                FROM UsoMedicamento
                WHERE DiasUso > 365
                );
SELECT * FROM Matriz;
    

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [19]:
UPDATE Matriz
    SET MedicamentosAcimaMedia = True
    WHERE ID IN (SELECT IDpessoa
        FROM MediaAux
        WHERE Qnt > (SELECT AVG(Qnt)
                 FROM MediaAux));
SELECT * FROM Matriz;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Verificar a média de medicamentos usados por pessoas Até 30 anos

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [20]:
CREATE VIEW PessoasAte30 AS
    SELECT IDpessoa U, COUNT(*) MedicamentosUsados
    FROM UsoMedicamento U, Pessoa P
    WHERE U.IDpessoa = P.ID
        AND P.Idade <= 30
    GROUP BY IDpessoa;

In [22]:
SELECT * FROM PessoasAte30;
SELECT AVG(MedicamentosUsados) FROM PessoasAte30;